In [1]:
import pennylane as qml
import tensorflow as tf
from pennylane import numpy as np
from matplotlib import pyplot as plt

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
def real_data(phi, **kwargs):
    # phi is a list with length 12
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.RX(phi[0], wires=0)
    qml.RY(phi[1], wires=0)
    qml.RZ(phi[2], wires=0)
    qml.RX(phi[3], wires=0)
    qml.RY(phi[4], wires=0)
    qml.RZ(phi[5], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RX(phi[6], wires=1)
    qml.RY(phi[7], wires=1)
    qml.RZ(phi[8], wires=1)
    qml.RX(phi[9], wires=1)
    qml.RY(phi[10], wires=1)
    qml.RZ(phi[11], wires=1)

In [3]:
# Wires 0 and 1 are the 2 qubit state. 

def generator(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    generator_layer(w[:11])
    generator_layer(w[11:22])
    generator_layer(w[22:33]) # includes w[22], doesnt include w[33]
    qml.RX(w[33], wires=0)
    qml.RY(w[34], wires=0)
    qml.RZ(w[35], wires=0)
    qml.RX(w[36], wires=0)
    qml.RY(w[37], wires=0)
    qml.RZ(w[38], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[39], wires=1)
    qml.RY(w[40], wires=1)
    qml.RZ(w[41], wires=1)
    qml.RX(w[42], wires=1)
    qml.RY(w[43], wires=1)
    qml.RZ(w[44], wires=1)

def generator_layer(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RX(w[2], wires=2)
    qml.RX(w[3], wires=3)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.RZ(w[6], wires=2)
    qml.RZ(w[7], wires=3)
    qml.MultiRZ(w[8], wires=[0, 1])
    qml.MultiRZ(w[9], wires=[2, 3])
    qml.MultiRZ(w[10], wires=[1, 2])
    # 11 weights

In [4]:
# the discriminator acts on wires 0, 1, and 4

def discriminator_layer(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RX(w[2], wires=4)
    qml.RZ(w[3], wires=0)
    qml.RZ(w[4], wires=1)
    qml.RZ(w[5], wires=4)
    qml.MultiRZ(w[6], wires=[0, 1])
    qml.MultiRZ(w[7], wires=[1, 4])

def discriminator(w, **kwargs):
    discriminator_layer(w[:8])
    discriminator_layer(w[8:16]) 
    discriminator_layer(w[16:32])
    qml.RX(w[32], wires=4)
    qml.RY(w[33], wires=4)
    qml.RZ(w[34], wires=4)

In [5]:
num_wires = 5
dev = qml.device('cirq.simulator', wires=num_wires)

In [6]:
@qml.qnode(dev, interface='tf')
def real_discriminator(phi, discriminator_weights):
    real_data(phi)
    discriminator(discriminator_weights)
    return qml.expval(qml.PauliZ(4))

@qml.qnode(dev, interface='tf')
def generator_discriminator(generator_weights, discriminator_weights):
    generator(generator_weights)
    discriminator(discriminator_weights)
    return qml.expval(qml.PauliZ(4))

In [7]:
def probability_real_real(discriminator_weights):
    # probability of guessing real data as real
    discriminator_output = real_discriminator(phi, discriminator_weights) # the output of the discriminator classifying the data as real
    probability_real_real = (discriminator_output + 1) / 2
    return probability_real_real

def probability_fake_real(generator_weights, discriminator_weights):
    # probability of guessing real fake as real
    # incorrect classification
    discriminator_output = generator_discriminator(generator_weights, discriminator_weights)
    probability_fake_real = (discriminator_output + 1) / 2
    return probability_fake_real
       
def discriminator_cost(discriminator_weights):
    accuracy = probability_real_real(discriminator_weights) - probability_fake_real(generator_weights, discriminator_weights)
    # accuracy = correct classification - incorrect classification
    cost = -accuracy    
    return cost

def generator_cost(generator_weights):
    accuracy = probability_fake_real(generator_weights, discriminator_weights)
    # accuracy = probability that the generator fools the discriminator
    cost = -accuracy
    return cost

In [8]:
# variables
phi = [np.pi] * 12
for i in range(len(phi)):
    phi[i] = phi[i] / np.random.randint(2, 11)
num_epochs = 30
eps = 1 

initial_generator_weights = np.array([np.pi] + [0] * 44) + \
    np.random.normal(scale=eps, size=(45,))
initial_discriminator_weights = np.random.normal(scale=eps, size=(35,))

generator_weights = tf.Variable(initial_generator_weights)
discriminator_weights = tf.Variable(initial_discriminator_weights)

Irrelevant question: where does the name 'eps' come from? epsilon?

In [9]:
# Let's run some tests first to make sure ur circuitry works.
# print(initial_discriminator_weights)
print(len(initial_discriminator_weights))
print('\nCost:', discriminator_cost(discriminator_weights).numpy())
print('\nProbability classify real as real: ', probability_real_real(generator_weights).numpy())
print('\nProbability classify fake as real: ', probability_fake_real(generator_weights, discriminator_weights).numpy())

35

Cost: -0.0034253858029842377

Probability classify real as real:  0.5809671878814697

Probability classify fake as real:  0.07850005850195885


In [10]:
opt = tf.keras.optimizers.SGD(0.4) # 0.4 is lr?

In [16]:
def train_discriminator():
    for epoch in range(num_epochs):
        cost = lambda: discriminator_cost(discriminator_weights) 
        # you need lambda because discriminator weights is a tensorflow object
        opt.minimize(cost, discriminator_weights)
        if epoch % 5 == 0:
            cost_val = discriminator_cost(discriminator_weights).numpy()
            print('Epoch  {}/{}, Cost: {}, Probability class real as real: {}'.format(epoch, num_epochs, cost_val, probability_real_real(discriminator_weights).numpy()))
        if epoch == num_epochs - 1:
            print('\n')

def train_generator():
    for epoch in range(num_epochs):
        cost = lambda: generator_cost(generator_weights)
        opt.minimize(cost, generator_weights)
        if epoch % 5 == 0:
            cost_val = generator_cost(generator_weights).numpy()
            print('Epoch  {}/{}, Cost: {}, Probability class fake as real: {}'.format(epoch, num_epochs, cost_val, probability_fake_real(generator_weights, discriminator_weights).numpy()))
        if epoch == num_epochs - 1:
            print('\n')

In [17]:
train_discriminator()
train_generator()

Epoch  0/30, Cost: -0.14299175143241882, Probability class real as real: 0.7159662395715714
Epoch  5/30, Cost: -0.16198615729808807, Probability class real as real: 0.6768261790275574
Epoch  10/30, Cost: -0.1782488077878952, Probability class real as real: 0.6543751358985901
Epoch  15/30, Cost: -0.1930706650018692, Probability class real as real: 0.6391105055809021
Epoch  20/30, Cost: -0.20613813400268555, Probability class real as real: 0.6282232850790024
Epoch  25/30, Cost: -0.2171032577753067, Probability class real as real: 0.6213741898536682


Epoch  0/30, Cost: -0.8329757153987885, Probability class fake as real: 0.8329757153987885
Epoch  5/30, Cost: -0.9646678753197193, Probability class fake as real: 0.9646678753197193
Epoch  10/30, Cost: -0.9865575945004821, Probability class fake as real: 0.9865575945004821
Epoch  15/30, Cost: -0.9927986208349466, Probability class fake as real: 0.9927986208349466
Epoch  20/30, Cost: -0.994613244663924, Probability class fake as real: 0.99461

In [18]:
probability_fake_real(generator_weights, discriminator_weights).numpy()

0.9953168514184654

In [19]:
print('Optimized discriminator weights\n', discriminator_weights.numpy())
print('\nInitial discriminator weights\n', initial_discriminator_weights)

Optimized discriminator weights
 [ 0.98678342 -0.43386663 -3.17111064  0.62260856  0.04342849  1.18291196
 -0.66513935 -1.21946626 -0.61854902 -1.01444608 -1.0025097  -0.06969379
 -0.11627456 -0.41911874 -0.62422002 -1.31842536  1.15465123  1.00688048
 -1.0759944  -1.20182703 -1.72498971 -0.15707318 -0.20518432  1.28586628
  1.59499949  1.35754469 -0.29718026 -1.24335632  1.57450562  0.97803845
 -0.34603064 -1.23144904  0.25517454 -1.42773793 -0.8975334 ]

Initial discriminator weights
 [ 0.98573627  0.65247181 -2.58823669  0.33964008 -0.87467339  0.92639282
  0.29698666 -0.36662934 -0.24270254 -0.2329808  -0.21704927  0.20564283
  0.0291995  -0.35253556 -0.49546954 -1.03084089  1.15465116  0.0640773
 -0.57206129 -1.20182703 -1.72498971 -0.21045368 -0.20518432  0.36111367
  1.59499949  1.35754469 -0.29718026 -1.24335632  1.57450562  0.97803845
 -0.34603064 -1.23144904  0.43664528 -0.50552278 -0.8975334 ]


In [18]:
probability_fake_real(generator_weights, discriminator_weights).numpy()

0.40479376912117004

In [21]:
print(generator_weights.numpy())
print('init weights', initial_generator_weights)

[ 3.90072086  0.90251517 -0.13497327 -0.9437307  -0.94730294  0.31199351
 -0.12415583 -1.26854345  0.29861301  2.00666326 -0.52638961 -1.02011791
  0.29098029 -1.24092841 -1.07395566  1.20113425 -0.60783449  1.81911011
  0.61488274 -0.2907043   0.27982617 -0.03171077  0.26173261  0.60752268
  1.10215144 -0.64240859  1.6822554  -0.01451248 -0.72422779  1.9995518
  0.55065701  2.8719099   0.03269038  1.68317262 -0.68062882  0.08401542
 -0.39987624 -0.18922004  1.85128989  1.06970315 -0.17565118  1.4433243
  0.54675685 -0.65416197  1.16924939]
init weights [ 3.87981050e+00  8.81604932e-01 -3.98517665e-01 -1.09227004e+00
 -1.43638646e+00  4.04968952e-02 -1.22335310e-01 -1.25580752e+00
  2.47595235e-01  1.89992169e+00 -9.35276970e-01 -9.93772849e-01
 -4.55457349e-02 -1.16198079e+00 -1.09638298e+00  9.12977741e-01
 -8.93065433e-01  1.82040040e+00  6.14882636e-01  2.64336199e-03
  3.06362859e-01 -5.16343454e-01  6.37671246e-01  4.10499432e-01
  1.14176107e+00 -6.42408602e-01  1.36818820e+00 -

In [25]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0), qml.PauliX(1), qml.PauliY(1), qml.PauliZ(1)] # oh this is the observation matrix

bloch_vector_real = qml.map(real_data, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real(phi)))
print("Generator Bloch vector: {}".format(bloch_vector_generator(generator_weights)))

# print('\n', generator_weights.numpy())
# print(phi)

Real Bloch vector: [ 4.33602571e-01  6.94913536e-01  2.38418579e-07  6.98950589e-02
 -2.59918928e-01 -7.73610234e-01]
Generator Bloch vector: [ 0.59286116  0.34908998  0.59910861 -0.57154025  0.25374156 -0.66421711]


In [23]:
@qml.qnode(dev)
def real_circuit(phi, **kwargs):
    real_data(phi)
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))
print(real_circuit(phi))

@qml.qnode(dev)
def generator_circuit(generator_weights, **kwargs):
    generator(generator_weights)
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))
print(generator_circuit(generator_weights))

[ 2.38418579e-07 -7.73610234e-01]
[ 0.59910861 -0.66421711]
